In [ ]:
%load_ext autoreload
%autoreload 2

In [12]:
%matplotlib osx

In [1]:
from full_prod_DRA import *
from buchi import buchi_from_ltl
# from Visualize import plot_buchi
import numpy as np
from env_sensing_error import CurrentWorld
import scipy
from plot_path_for_prod import *
from graphviz import Source
from Plot_Path import *

/Users/gaoqitong/anaconda/envs/py27/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
def get_global_opt(ltl, env, wfts, region_list):
    ltl = ltl.lower()

    region_list = update_wfts_ap(region_list)

    wfts.add_initial(region_list[np.ravel_multi_index(env.start_coord, env.shape[:-1])])


    # buchi = buchi_from_ltl(ltl,None)
    # my_buchi = Buchi_Automaton(buchi)

    rabin = env.rabin

    full_prod = FullProd(wfts, rabin)
    full_prod.construct_fullproduct()
    count = 0
    for i in full_prod.states:
        for j in full_prod.transition[i].keys():
            if full_prod.transition[i][j] is not None:
                count += 1 

    opt=search_opt_run(full_prod)
    opt_path = [tuple(list(opt[0][i][0].coord) + [opt[0][i][1]]) for i in range(len(opt[0]))]
    print 'Plan synthesized:'+str(opt_path)
    return opt_path, region_list, wfts

def update_wfts_ap(region_list):
    # Update sensed ap for all regions
    
    last_coord_dict_extra = set((k,tuple(v)) for k,v in env.last_dynamic_coord_dict.items()) - set((k,tuple(v)) for k,v in env.dynamic_coord_dict.items())
    new_coord_dict_extra = set((k,tuple(v)) for k,v in env.dynamic_coord_dict.items()) - set((k,tuple(v)) for k,v in env.last_dynamic_coord_dict.items())
    
    for k,v in last_coord_dict_extra:
        print "k",k
        for i in v:
            print "i",i
            region_list[np.ravel_multi_index(k, (10,10))].app.remove(i.lower())
            region_list[np.ravel_multi_index(k, (10,10))].ap.remove(i.lower())

    for k,v in new_coord_dict_extra:
        for i in v:
            region_list[np.ravel_multi_index(k, (10,10))].app += [i.lower()]
            region_list[np.ravel_multi_index(k, (10,10))].ap += [i.lower()]
    return region_list

def get_opt_rabin(opt_path):
    opt_rabin = []
    for i in opt_path:
        if len(opt_rabin) == 0:
            opt_rabin += [i[-1]]
        if i[-1] != opt_rabin[-1]:
            opt_rabin += [i[-1]]
    return opt_rabin

def get_next_ltl(current_rabin, opt_rabin):
    # input 'current_rabin' is with type int
    current_rabin = str(current_rabin)
    current_idx = np.where(np.array(opt_rabin) == np.int(current_rabin))[0][0]
    next_rabin = str(opt_rabin[current_idx+1])
    ltl_list = [i["label"][1:] for i in env.rabin.graph[current_rabin][next_rabin].values()]
    ltl = ")||<>(".join(ltl_list)
    ltl = "<>(" + ltl + ")"
    return ltl

In [ ]:
# Example Used in DQN Paper

# ltl = "<>(A && <>(B && <>T)) && []<>A && []<>B && []!C".lower()
# ltl = "<>(A && <>(B && <>T)) && []!C".lower()
ltl = "<>(A && <>B) && []!C".lower()
env = CurrentWorld(ltl)

region_list = ["r"+str(i) for i in range(100)]
for i in range(len(region_list)):
    coord = np.unravel_index(i, (10,10))
    # Use static dict here for original DQN paper example
    if len(env.dynamic_coord_dict[coord]) > 0:
        region_list[i] = Region(coord, [ap.lower() for ap in env.dynamic_coord_dict[coord]], region_list[i])
    else:
        region_list[i] = Region(coord, [], region_list[i])
    

wfts = wFTS(set(), {}, set())
for i in region_list:
    wfts.add_states(i)
    
for i in region_list:
    current_coord = list(i.coord)
    candidates = [np.array(current_coord), np.add(current_coord,[0,1]), np.add(current_coord,[0,-1]), 
                  np.add(current_coord,[1,0]), np.add(current_coord,[-1,0])]
    candidates = [np.ravel_multi_index(c, env.shape[:-1]) for c in candidates if c[0]>=0 and c[1]>=0 and c[0]<env.shape[0] and c[1]<env.shape[1]]
    for c in candidates:
        wfts.add_transition(i, region_list[c], 1)
            
wfts.add_initial(region_list[np.ravel_multi_index(env.start_coord, env.shape[:-1])])
    

# buchi = buchi_from_ltl(ltl,None)
# my_buchi = Buchi_Automaton(buchi)

rabin = env.rabin

full_prod = FullProd(wfts, rabin)
full_prod.construct_fullproduct()
count = 0
for i in full_prod.states:
    for j in full_prod.transition[i].keys():
        if full_prod.transition[i][j] is not None:
            count += 1 

opt=search_opt_run(full_prod)
opt_path = [tuple(list(opt[0][i][0].coord) + [opt[0][i][1]]) for i in range(len(opt[0]))]
print 'Plan synthesized:'+str(opt_path)

In [15]:
ltl = "<>(A && <>B) && []!C".lower()
env = CurrentWorld(ltl)
prod_plan = Prod_Planning(env, ltl)

0
0
reset


In [ ]:
prod_plan.full_prod.states

In [ ]:
prod_plan.env.rabin.states

In [20]:
print prod_plan.get_global_opt()

Plan synthesized:[(4, 1, 1), (3, 1, 1), (2, 1, 1), (1, 1, 1), (1, 2, 1), (1, 3, 1), (2, 3, 1), (2, 4, 1), (2, 5, 1), (2, 6, 1), (2, 7, 1), (3, 7, 2), (4, 7, 2), (5, 7, 2), (6, 7, 2), (7, 7, 2), (7, 6, 2), (7, 5, 2), (6, 5, 2), (6, 4, 2), (6, 3, 2), (6, 2, 2), (5, 2, 2), (5, 3, 3), (5, 3, 4), (5, 3, 4), (5, 3, 4)]
None


In [17]:
prod_plan.get_opt_rabin()

In [18]:
print prod_plan.opt_rabin

[1, 2, 3, 4]


In [11]:
print prod_plan.get_next_ltl(2)

<>(!a&b&!c)||<>(a&b&!c)


In [23]:
convert_path_to_action(prod_plan.get_local_opt((5,7), "<>(!a&b&!c)||<>(a&b&!c)"))

0
0
Local Plan synthesized:[(5, 7, 0), (5, 6, 0), (5, 5, 0), (5, 4, 0), (5, 3, 0), (5, 2, 0), (6, 2, 1), (6, 2, 1), (6, 2, 1)]


0

In [ ]:
prod_plan.region_list[np.ravel_multi_index((5,2),(10,10))].ap

In [ ]:
prod_plan.env.last_ap_dict

In [ ]:
prod_plan.env.ap_dict

In [22]:
def convert_path_to_action(path):
    current_coord = path[0]
    next_coord = path[1]
    if current_coord[0] == next_coord[0] and current_coord[1]-1 == next_coord[1]:
        return 0
    if current_coord[0]+1 == next_coord[0] and current_coord[1] == next_coord[1]:
        return 1
    if current_coord[0] == next_coord[0] and current_coord[1]+1 == next_coord[1]:
        return 2
    if current_coord[0]-1 == next_coord[0] and current_coord[1] == next_coord[1]:
        return 3
    if current_coord[0] == next_coord[0] and current_coord[1] == next_coord[1]:
        return 4
    

In [14]:
class Prod_Planning(object):
    def __init__(self, env, ltl):
        self.ltl = ltl.lower()
        self.env = env
        
        region_list = ["r"+str(i) for i in range(100)]
        for i in range(len(region_list)):
            coord = np.unravel_index(i, (env.shape[0],env.shape[1]))
            if len(env.dynamic_coord_dict[coord]) >0:
                region_list[i] = Region(coord, [ap.lower() for ap in env.dynamic_coord_dict[coord]], region_list[i])
            else:
                region_list[i] = Region(coord, [], region_list[i])
                
        wfts = wFTS(set(), {}, set())
        for i in region_list:
            wfts.add_states(i)

        for i in region_list:
            current_coord = list(i.coord)
            candidates = [np.array(current_coord), np.add(current_coord,[0,1]), np.add(current_coord,[0,-1]), 
                          np.add(current_coord,[1,0]), np.add(current_coord,[-1,0])]
            candidates = [np.ravel_multi_index(c, env.shape[:-1]) for c in candidates if c[0]>=0 and c[1]>=0 and c[0]<env.shape[0] and c[1]<env.shape[1]]
            for c in candidates:
                wfts.add_transition(i, region_list[c], 1)

        wfts.add_initial(region_list[np.ravel_multi_index(env.start_coord, env.shape[:-1])])
        
        self.region_list = region_list
        self.wfts = wfts
        self.opt_path = []
        
    def update_wfts_ap(self):
        last_coord_dict_extra = set((k,tuple(v)) for k,v in self.env.last_dynamic_coord_dict.items()) - set((k,tuple(v)) for k,v in self.env.dynamic_coord_dict.items())
        new_coord_dict_extra = set((k,tuple(v)) for k,v in self.env.dynamic_coord_dict.items()) - set((k,tuple(v)) for k,v in self.env.last_dynamic_coord_dict.items())
#         print "last_extra: ", last_coord_dict_extra
#         print "new_extra: ", new_coord_dict_extra
        for k,v in last_coord_dict_extra:
            print "k",k
            for i in v:
                print "i",i
                self.region_list[np.ravel_multi_index(k, (10,10))].app.remove(i.lower())
                self.region_list[np.ravel_multi_index(k, (10,10))].ap.remove(i.lower())

        for k,v in new_coord_dict_extra:
            for i in v:
                self.region_list[np.ravel_multi_index(k, (10,10))].app += [i.lower()]
                self.region_list[np.ravel_multi_index(k, (10,10))].ap += [i.lower()]
    
    def get_global_opt(self):
        opt_path = []

#         self.region_list = self.update_wfts_ap()
        self.wfts.replace_initial(self.region_list[np.ravel_multi_index(self.env.start_coord, self.env.shape[:-1])])
        
        rabin = self.env.rabin
        
        full_prod = FullProd(self.wfts, rabin)
        self.full_prod = full_prod
        full_prod.construct_fullproduct()
        count = 0
        for i in full_prod.states:
            for j in full_prod.transition[i].keys():
                if full_prod.transition[i][j] is not None:
                    count += 1 
                    
        # while len(opt_path) == 0:
        try:
            opt=search_opt_run(full_prod)
            opt_path = [tuple(list(opt[0][i][0].coord) + [opt[0][i][1]]) for i in range(len(opt[0]))]
            self.opt_path = opt_path
        except:
            pass
        
        # print 'Plan synthesized:'+str(opt_path)
#         return self.opt_path, self.region_list, self.wfts

    def get_opt_rabin(self):
        opt_rabin = []
        for i in self.opt_path:
            if len(opt_rabin) == 0:
                opt_rabin += [i[-1]]
            if i[-1] != opt_rabin[-1]:
                opt_rabin += [i[-1]]
#         return opt_rabin
        self.opt_rabin = opt_rabin
    
    def get_next_ltl(self, current_rabin):
        # input 'current_rabin' is with type int
        current_rabin = str(current_rabin)
        current_idx = np.where(np.array(self.opt_rabin) == np.int(current_rabin))[0][0]
        next_rabin = str(self.opt_rabin[current_idx+1])
        ltl_list = [i["label"][1:] for i in self.env.rabin.graph[current_rabin][next_rabin].values()]
        next_ltl = ")||<>(".join(ltl_list)
        next_ltl = "<>(" + next_ltl + ")"
        return next_ltl
    
    def get_local_opt(self, new_start_coord, new_ltl):
        opt_local_path = []

        rabin = Rabin_Automaton(new_ltl, env.dynamic_coord_dict)
        self.wfts.replace_initial(self.region_list[np.ravel_multi_index(new_start_coord, env.shape[:-1])])
        
        full_prod = FullProd(self.wfts, rabin)
        full_prod.construct_fullproduct()
        count = 0
        for i in full_prod.states:
            for j in full_prod.transition[i].keys():
                if full_prod.transition[i][j] is not None:
                    count += 1 
                    
        # while len(opt_local_path) == 0:
        try:
            opt=search_opt_run(full_prod)
            opt_local_path = [tuple(list(opt[0][i][0].coord) + [opt[0][i][1]]) for i in range(len(opt[0]))]
            return opt_local_path
        except:
            return None
        # print 'Local Plan synthesized:'+str(opt_local_path)

In [ ]:
get_global_opt(ltl,env,wfts,region_list)

In [ ]:
env.step()

In [ ]:
# For Guide Learning

# wfts.replace_initial(region_list[np.ravel_multi_index(ns[:2],(10,10))])
    
# buchi = buchi_from_ltl(ltl,None)
# my_buchi = Buchi_Automaton(buchi)
new_ltl = get_next_ltl("2", get_opt_rabin(opt_path))

rabin = Rabin_Automaton(new_ltl, env.dynamic_coord_dict)

full_prod = FullProd(wfts, rabin)
full_prod.construct_fullproduct()
count = 0
for i in full_prod.states:
    for j in full_prod.transition[i].keys():
        if full_prod.transition[i][j] is not None:
            count += 1 

opt=search_opt_run(full_prod)
opt_path = [tuple(list(opt[0][i][0].coord) + [opt[0][i][1]]) for i in range(len(opt[0]))]
print 'Plan synthesized:'+str(opt_path)

In [ ]:
plt.close("all")

In [ ]:
env.last_ap_dict

In [ ]:
env.ap_dict

In [ ]:
env.last_dynamic_coord_dict[(2,7)]

In [ ]:
env.dynamic_coord_dict[(2,7)]

In [ ]:
region_list[np.ravel_multi_index((6,6),(10,10))].ap

In [ ]:
set((k,tuple(v)) for k,v in env.last_dynamic_coord_dict.items()) - set((k,tuple(v)) for k,v in env.dynamic_coord_dict.items())

In [ ]:
env.reset()

In [ ]:
ltl = "<>(A && <>B) && []!C".lower()
env = CurrentWorld(ltl)
region_list = ["r"+str(i) for i in range(100)]
for i in range(len(region_list)):
    coord = np.unravel_index(i, (10,10))
    # Use static dict here for original DQN paper example
    if len(env.dynamic_coord_dict[coord]) > 0:
        region_list[i] = Region(coord, [ap.lower() for ap in env.dynamic_coord_dict[coord]], region_list[i])
    else:
        region_list[i] = Region(coord, [], region_list[i])
env.reset()
region_list = update_wfts_ap(region_list)

In [ ]:
plt.close("all")

In [ ]:
env.reset()
region_list = update_wfts_ap(region_list)
np.random.seed(100)
for i in range(10):
    a = np.random.randint(env.nA)
    ns,r,d,_ = env.step(a)
    ns = np.unravel_index(ns,(10,10,5))
    ns_plot = ns[:2]
    render(env,ns_plot,a)
    region_list = update_wfts_ap(region_list)
    

In [ ]:
env.dynamic_coord_dict[(6,6)]

In [ ]:
env.ap_dict

In [ ]:
new_ltl = get_next_ltl("2",get_opt_rabin(opt_path))

In [ ]:
new_ltl

In [ ]:
locate_next_rabin("2")

In [ ]:
[i["label"] for i in env.rabin.graph["2"]["3"].values()]

In [ ]:
from collections import defaultdict

In [ ]:
# Generate Policy from Opt Path

dra_policy = {}
for i in opt_path:
    # blocks in down/left/up/right position
    dra_policy[tuple(np.add(list(i), [1,0,0]))] = 0
    dra_policy[tuple(np.add(list(i), [0,-1,0]))] = 1
    dra_policy[tuple(np.add(list(i), [-1,0,0]))] = 2
    dra_policy[tuple(np.add(list(i), [0,1,0]))] = 3
    # blocks in down-right/down-left/up-left/up-right position
    dra_policy[tuple(np.add(list(i), [1,1,0]))] = 0
    dra_policy[tuple(np.add(list(i), [1,-1,0]))] = 1
    dra_policy[tuple(np.add(list(i), [-1,-1,0]))] = 2
    dra_policy[tuple(np.add(list(i), [-1,1,0]))] = 3

In [ ]:
print region_list[0].apr[0]

In [ ]:
dra_policy.keys()

In [ ]:
with open("current_buchi_dot.txt", "r") as dotfile:
    text = dotfile.read()
Source(text)

In [ ]:
with open("my.dot", "r") as dotfile:
    text = dotfile.read()
Source(text)

In [ ]:
import pydot as pd
graph = pd.graph_from_dot_file("my.dot")
graph = graph.pop()
graph.write_pdf("mydot.pdf")

In [ ]:
plot_path_for_prod(env, ltl, opt_path)

In [ ]:
plt.close("all")

In [ ]:
# Example

r4 = Region((0,2),['basket'],'r4')
r5 = Region((1,2),['rball'],'r5')
r6 = Region((2,2),[],'r6')
c1 = Region((0,1),[],'c1')
c2 = Region((1,1),[],'c2')
c3 = Region((2,1),[],'c3')
r1 = Region((0,0),[],'r1')
r2 = Region((1,0),['basket'],'r2')
r3 = Region((2,0),['gball'],'r3')

region_list = [r1, r2, r3, r4, r5, r6, c1, c2, c3]

wfts = wFTS()

for i in region_list:
    wfts.add_states(i)
    
wfts.add_transition(r1,c1,1)
wfts.add_transition(c1,c2,1)
wfts.add_transition(c1,r4,1)
wfts.add_transition(c2,c3,1)
wfts.add_transition(c2,r2,1)
wfts.add_transition(c2,r5,1)
wfts.add_transition(c3,r6,1)
wfts.add_transition(c3,r3,1)

wfts.add_initial(r1)

formula = '<>(rball && <>basket) && <>[]r1'
buchi = buchi_from_ltl(formula,None)
my_buchi = Buchi_Automaton(buchi)

full_prod = FullProd(wfts,my_buchi)
full_prod.construct_fullproduct()
count = 0
for i in full_prod.states:
    for j in full_prod.transition[i].keys():
        if full_prod.transition[i][j] is not None:
            count += 1 

opt=search_opt_run(full_prod)
print 'Plan synthesized:'+str([opt[0][i][0] for i in range(len(opt[0]))])

In [ ]:
#plot buchi
my_buchi.plot()

#plot full product
full_prod_graph = full_prod.return_graph()
plot(full_prod_graph,'full_prod')